In [ ]:
!pip install configparser
!pip install pandas
import tweepy
import pandas as pd
import numpy as np
import requests
import os
import re
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def filter(data):
  df = data
  df = df[df["tweet"].str.contains(" crypto | cryptocurrency | bitcoin | btc ")==True]
  return df
def clean(text):
  text = re.sub(r'@[A-Za-z0-9:]+', '',text) #removes @ mentions
  text = re.sub(r'#','',text) #remove #
  text = re.sub(r'RT[\s]+','',text) # remove RT
  text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text) #removes links
  text = re.sub(r'\s*$','',text) #remove leading and trailing whitespace
  text = re.sub(r'^\s*','',text) #remove leading and leading whitespace
  text = re.sub(r'.+:', '',text) #remove username:
  return text

def clean_data(data):
  #data = data[data.language == 'en']
  data.replace("", np.nan, inplace=True) # replace empty string to nan 
  data.dropna(subset = ["tweet"], inplace=True) #remove all rows with nan tweet
  data['tweet'] = data['tweet'].apply(clean)
  data.replace("", np.nan, inplace=True) # replace empty string to nan 
  data.dropna(subset = ["tweet"], inplace=True) #remove all rows with nan twee
  data = data.drop_duplicates(subset = 'tweet', keep = False ) # remove duplicates and keep none 
  return data

#t = df_gold.iloc[0]['tweet']


In [ ]:
df_raw_ = pd.read_csv('/content/drive/MyDrive/fyp/fyp2/tweets16m.csv', delimiter=';', skiprows=0, lineterminator='\n' )
df_raw_.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,user,fullname,url,timestamp,replies,likes,retweets,text\r
0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,1.132977e+18,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasn't caught in the s...
3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...


In [ ]:
df_raw = df_raw_.copy()
# Rename columns
df_raw.columns = ["id", "user", "fullname", "url", "timestamp", "replies","likes","retweets","text"]

# Show the first 5 rows of the dataframe.
# You can specify the number of rows to be shown as follows: df_raw.head(10)
df_raw.head()

,id,user,fullname,url,timestamp,replies,likes,retweets,text
0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,1.132977e+18,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasn't caught in the s...
3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...


In [ ]:
df = df_raw[['timestamp','text']]
df.sample(5)

,timestamp,text
880022,2019-05-20 11:14:15+00,Bitcoin Price Hits All-Time Highs in These Cou...
9357903,2019-07-27 12:07:08+00,Confessions of a Venezuelan Bitcoin miner http...
5066663,2019-06-01 10:42:59+00,@agasp68 1 btc already. thanks eIon!😊\r
11026335,2019-08-21 00:44:14+00,Yeah new bit currency\r
12563544,2019-09-13 15:11:09+00,Interested on buying Bitcoin with direct bank ...


In [ ]:
#Create split by dates 
df.timestamp.dtypes

dtype('O')

In [ ]:
df['date'] = pd.to_datetime(df['timestamp'],format= '%Y-%m-%d').dt.date
df['date']

df = df.sort_values(by='date')
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,timestamp,text,date
11291156,2007-04-19 07:14:38+00,is happily mugging at BTC where she will hook ...,2007-04-19
21,2009-01-11 03:33:52+00,Running bitcoin\r,2009-01-11
5164467,2009-01-21 17:29:40+00,Looking at ways to add more anonymity to bitco...,2009-01-21
5210398,2009-01-27 20:14:10+00,Thinking about how to reduce CO2 emissions fro...,2009-01-27
5269667,2009-01-29 13:37:53+00,From: Satoshi Nakamoto - 2009-01-11 22:32 Bitc...,2009-01-29


In [ ]:
df = df.set_index('date')
df.head()

,timestamp,text
date,,
2007-04-19,2007-04-19 07:14:38+00,is happily mugging at BTC where she will hook ...
2009-01-11,2009-01-11 03:33:52+00,Running bitcoin\r
2009-01-21,2009-01-21 17:29:40+00,Looking at ways to add more anonymity to bitco...
2009-01-27,2009-01-27 20:14:10+00,Thinking about how to reduce CO2 emissions fro...
2009-01-29,2009-01-29 13:37:53+00,From: Satoshi Nakamoto - 2009-01-11 22:32 Bitc...


In [ ]:
# df.date.year > '2017'
startdate = pd.to_datetime("2017-01-01").date()
df = df.loc[startdate:]
df.tail()

,timestamp,text
date,,
2019-11-23,2019-11-23 05:47:01+00,Bitcoin:$7231.27732436\nEthereum:$150.48909214...
2019-11-23,2019-11-23 05:40:05+00,BTC\n\nfinexの乖離がまだ異常水域です。\n\nペナントの収束タイミングに合わせた...
2019-11-23,2019-11-23 05:40:05+00,"#BTC New Hub, we have relaunch our #NEWYORKCOI..."
2019-11-23,2019-11-23 05:39:55+00,トランザクション収入がまともにないBitcoinさんのマイナーは単価に依存するからこうなるん...
2019-11-23,2019-11-23 15:45:06+00,$BTC - an update on the longer term view for B...


In [ ]:
!pip install whatthelang

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 786 kB 1.4 MB/s 
     |████████████████████████████████| 871 kB 53.7 MB/s 
     |████████████████████████████████| 244 kB 68.6 MB/s 
  Created wheel for whatthelang: filename=whatthelang-1.0.1-py3-none-any.whl size=789818 sha256=39c91983a89576e04f38871e5e5645d0ee8cc170de78950f3b6c554f4ee81532
  Stored in directory: /root/.cache/pip/wheels/91/5b/fe/43b4b1eb6511aed55a9c5ecf3a178a9b06d783a58907dbc87b
  Created wheel for pyfasttext: filename=pyfasttext-0.4.6-cp37-cp37m-linux_x86_64.whl size=1461363 sha256=8f0d4b5f805d43c858033f096b9ea8821b1bb922277d208560eb8be8c2724b63
  Stored in directory: /root/.cache/pip/wheels/e6/e7/60/c4506ea3173416f3774e221bd741d06bfc2127ec23ceff6619
Successfully built whatthelang pyfasttext


In [ ]:
from whatthelang import WhatTheLang
wtl = WhatTheLang()
result = [wtl.predict_lang(row) for row in df['text']]
df['lang'] = result
df.head()

,timestamp,text,lang
date,,,
2017-01-01,2017-01-01 10:30:10+00,$975.00 at 11:30 UTC [24h Range: $950.81 - $97...,en
2017-01-01,2017-01-01 10:30:06+00,#Bitcoin\nUltima: R$ 3440.00 Alta: R$ 3440.00 ...,en
2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00@bitstamp. High $...,en
2017-01-01,2017-01-01 10:30:06+00,#8BitCoin #8BIT $0.011464 (0.86%) 0.00001195 B...,en
2017-01-01,2017-01-01 10:23:44+00,#Bitcoin last trade\n\n@bitfinex $976.81\n@btc...,en


In [ ]:
#ATTENTION, we will take only text, date and lang column FOR THE texts in ENglish and below DATES starting 2017 jan
en_df = df[df["lang"] == 'en']
en_df.sample(10, random_state = 5)

,timestamp,text,lang
date,,,
2019-10-21,2019-10-21 08:31:09+00,Chainlink is gaining fast.\n\nhttps://t.co/LQS...,en
2019-06-21,2019-06-21 21:02:12+00,Bitcoin turned 10 in January. A decade after S...,en
2019-06-07,2019-06-07 09:15:22+00,RT wiz: I can see a future where everyone has ...,en
2019-09-19,2019-09-19 14:28:59+00,If Bitcoin ETF's are being rejected.. what are...,en
2019-09-15,2019-09-15 20:52:33+00,@NYC | Immediate Hire - Customer Service Oppor...,en
2018-03-26,2018-03-26 20:07:47+00,#BTC Average: 7981.22$\n\n#Bitfinex - 7910.00$...,en
2019-09-20,2019-09-20 18:01:18+00,₿ Actual #Bitcoin #BTC price is \n1 BTC = 9.20...,en
2019-11-06,2019-11-06 17:06:30+00,Explainer: Central Bank Digital Currencies: Ed...,en
2019-05-16,2019-05-16 03:30:39+00,https://t.co/6Szi9FmjcK\n\nBTC/USD - HEDEF BÖL...,en


In [ ]:
# checking for null values, if any
en_df.isnull().sum()

timestamp    0
text         0
lang         0
dtype: int64

In [ ]:
#ditching all row when text is null, as need text for analysis
en_df.dropna(how='any', inplace=True)
en_df.sample(3)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,timestamp,text,lang
date,,,
2019-11-04,2019-11-04 09:01:04+00,Cardano $ADA Sell Signal\nCurrent price: 0.000...,en
2019-06-23,2019-06-23 22:31:50+00,New Free Bitcoin 2019 - 2020. 30.000 Satoshi p...,en
2019-08-31,2019-08-31 17:59:01+00,btc wallet - Binance https://t.co/zmW44accor\r,en


In [ ]:
en_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12343033 entries, 2017-01-01 to 2019-11-23
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  object
 1   text       object
 2   lang       object
dtypes: object(3)
memory usage: 376.7+ MB


In [ ]:
en_df.head()

,timestamp,text,lang
date,,,
2017-01-01,2017-01-01 10:30:10+00,$975.00 at 11:30 UTC [24h Range: $950.81 - $97...,en
2017-01-01,2017-01-01 10:30:06+00,#Bitcoin\nUltima: R$ 3440.00 Alta: R$ 3440.00 ...,en
2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00@bitstamp. High $...,en
2017-01-01,2017-01-01 10:30:06+00,#8BitCoin #8BIT $0.011464 (0.86%) 0.00001195 B...,en
2017-01-01,2017-01-01 10:23:44+00,#Bitcoin last trade\n\n@bitfinex $976.81\n@btc...,en


In [ ]:
#remove retweets and clean
df = en_df.copy()
df = df.rename({'text': 'tweet', 'timestamp': 'date'}, axis='columns')
#df = df[['date', 'tweet', 'is_retweet']]
df = df.replace("", np.nan)
df = df.dropna()
#df = df.drop(df[df.is_retweet == True].index)
df = clean_data(df)

display(df)
df = df.drop_duplicates(subset = 'tweet', keep = False )
display(df)


,date,tweet,lang
date,,,
2017-01-01,2017-01-01 10:30:10+00,2814 BTC],en
2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00. High $975.00. L...,en
2017-01-01,2017-01-01 10:30:06+00,8BitCoin 8BIT $0.011464 (0.86%) 0.00001195 BTC...,en
2017-01-01,2017-01-01 10:23:44+00,Bitcoin last trade\n\n $976.81\n $939.00\n\nSe...,en
2017-01-01,2017-01-01 10:30:20+00,$977.38 bitfinex;\n$974.98 GDAX;\n$975.00 bits...,en
...,...,...,...
2019-11-23,2019-11-23 05:40:39+00,"Bitcoin price 7238.65641481, DOWN -4.74 percen...",en
2019-11-23,2019-11-23 05:40:02+00,A $XMR is worth 0.00696045 BTC,en
2019-11-23,2019-11-23 05:40:03+00,"Top 5 cryptocurrencies \n03\n $7,234.439\n $15...",en


,date,tweet,lang
date,,,
2017-01-01,2017-01-01 10:30:10+00,2814 BTC],en
2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00. High $975.00. L...,en
2017-01-01,2017-01-01 10:30:06+00,8BitCoin 8BIT $0.011464 (0.86%) 0.00001195 BTC...,en
2017-01-01,2017-01-01 10:23:44+00,Bitcoin last trade\n\n $976.81\n $939.00\n\nSe...,en
2017-01-01,2017-01-01 10:30:20+00,$977.38 bitfinex;\n$974.98 GDAX;\n$975.00 bits...,en
...,...,...,...
2019-11-23,2019-11-23 05:40:39+00,"Bitcoin price 7238.65641481, DOWN -4.74 percen...",en
2019-11-23,2019-11-23 05:40:02+00,A $XMR is worth 0.00696045 BTC,en
2019-11-23,2019-11-23 05:40:03+00,"Top 5 cryptocurrencies \n03\n $7,234.439\n $15...",en


In [ ]:
#display(df)

df.to_csv('/content/drive/MyDrive/fyp/fyp2/tweets16m_cleaned.csv.csv')

In [ ]:
##clean bitcoin daily example 

df_o = pd.read_csv('/content/drive/MyDrive/fyp/fyp2/BTC_tweets_daily_example.csv')
display(df_o)

,Unnamed: 0,Date,Tweet,Screen_name,Source,Link,Sentiment,sent_score,New_Sentiment_Score,New_Sentiment_State
0,0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0.0,0.000000,0.0
1,1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral'],0.0,0.000000,0.0
2,2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.136364,1.0
3,3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.400000,1.0
4,4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,[],"<a href=""http://twitter.com/download/android"" ...",['positive'],1.0,0.468182,1.0
...,...,...,...,...,...,...,...,...,...,...
50868,50854,Fri Mar 23 08:55:16 +0000 2018,RT @fixy_app: Fixy Network brings popular cryp...,quoting_lives,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.600000,1.0
50869,50855,Fri Mar 23 08:55:17 +0000 2018,RT @bethereumteam: After a successful launch o...,VariPewitt,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.375000,1.0
50870,50856,Fri Mar 23 08:55:18 +0000 2018,"RT @GymRewards: Buy #GYMRewards Tokens, Bonus ...",urbancoinerz,"[u'GYMRewards', u'ICO', u'cryptocurrency', u'm...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0.0,0.000000,0.0
50871,50857,Fri Mar 23 08:55:19 +0000 2018,I added a video to a @YouTube playlist https:/...,MRDanishShahab,[],"<a href=""http://www.google.com/"" rel=""nofollow...",['positive'],1.0,0.400000,1.0


In [ ]:
df = df_o.copy()
df = df[['Date','Tweet','Sentiment','sent_score']]
df = df.rename({'Date': 'date', 'Tweet': 'tweet', 'Sentiment': 'sentiment'}, axis='columns')
display(df)
df = clean_data(df)
display(df)


,date,tweet,sentiment,sent_score
0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",['neutral'],0.0
1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,['neutral'],0.0
2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,['positive'],1.0
3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,['positive'],1.0
4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,['positive'],1.0
...,...,...,...,...
50868,Fri Mar 23 08:55:16 +0000 2018,RT @fixy_app: Fixy Network brings popular cryp...,['positive'],1.0
50869,Fri Mar 23 08:55:17 +0000 2018,RT @bethereumteam: After a successful launch o...,['positive'],1.0
50870,Fri Mar 23 08:55:18 +0000 2018,"RT @GymRewards: Buy #GYMRewards Tokens, Bonus ...",['neutral'],0.0
50871,Fri Mar 23 08:55:19 +0000 2018,I added a video to a @YouTube playlist https:/...,['positive'],1.0


,date,tweet,sentiment,sent_score
0,Fri Mar 23 00:40:32 +0000 2018,htt…,['neutral'],0.0
1,Fri Mar 23 00:40:34 +0000 2018,@_Kevin_Pham _sage But (dum b a ss) said you...,['neutral'],0.0
2,Fri Mar 23 00:40:35 +0000 2018,Another use case for blockchain and Tipper. Th...,['positive'],1.0
3,Fri Mar 23 00:40:36 +0000 2018,free coins,['positive'],1.0
4,Fri Mar 23 00:40:36 +0000 2018,WE are happy to announce that PayVX Presale Ph...,['positive'],1.0
...,...,...,...,...
50868,Fri Mar 23 08:55:16 +0000 2018,Fixy Network brings popular cryptocurrencies ...,['positive'],1.0
50869,Fri Mar 23 08:55:17 +0000 2018,//t…,['positive'],1.0
50870,Fri Mar 23 08:55:18 +0000 2018,"Buy GYMRewards Tokens, Bonus Time is ending! ,...",['neutral'],0.0
50871,Fri Mar 23 08:55:19 +0000 2018,I added a video to a playlist How To Bitcoin...,['positive'],1.0


In [ ]:
df.to_csv('/content/drive/MyDrive/fyp/fyp2/BTC_tweets_daily_example_cleaned.csv')

In [ ]:
#clean gold label 

input = '/content/drive/MyDrive/fyp/fyp2/data with sentient scores/manual_label_Gold.csv'
name = 'manual_label_Gold_cleaned'
name = name + '.csv'
print(name)
df_gold = pd.read_csv(input)

display(df_gold)

manual_label_Gold_cleaned.csv


,Unnamed: 0,index,date,tweet,is_retweet,real_score
0,0,0,2021-11-18 19:52:08,🔥🔥🔥 ANOTHER MASSIVE WIN! 🔥🔥🔥\n\nLUNA/USD Take-...,False,1.00
1,1,1,2021-10-23 09:37:54,&gt;There is no inflation\n&gt;There is some i...,False,0.00
2,2,2,2022-04-19 04:12:41,China’s largest broadcaster’s NFT series becom...,False,0.00
3,3,3,2021-05-29 22:54:12,❗Bee Network-Cloud Mining on Your Phone ❗☁️⛏️💰...,False,0.00
4,4,4,2022-03-18 21:04:16,"Have you heard of $PIT, one of the largest com...",False,0.25
...,...,...,...,...,...,...
295,295,295,2022-01-12 09:21:10,NFT news.\n\nKakao Entertainment to expand its...,False,0.00
296,296,296,2021-04-24 12:24:23,Crypto mining as it should be! \n\nZionodes pr...,False,0.00
297,297,297,2021-07-21 04:19:31,"The state now has 1,350 active locally-acquire...",False,0.00
298,298,298,2021-10-19 17:25:28,What’s amazing it’s not just for the rich and ...,False,0.25


In [ ]:
df_gold_cleaned = clean_data(df_gold)

display(df_gold_cleaned)

df_gold_cleaned.to_csv('/content/drive/MyDrive/fyp/fyp2/cleaned data/' +  name)

,Unnamed: 0,index,date,tweet,is_retweet,real_score
0,0,0,2021-11-18 19:52:08,🔥🔥🔥 ANOTHER MASSIVE WIN! 🔥🔥🔥\n\nLUNA/USD Take-...,False,1.00
1,1,1,2021-10-23 09:37:54,&gt;There is no inflation\n&gt;There is some i...,False,0.00
2,2,2,2022-04-19 04:12:41,China’s largest broadcaster’s NFT series becom...,False,0.00
3,3,3,2021-05-29 22:54:12,❗Bee Network-Cloud Mining on Your Phone ❗☁️⛏️💰...,False,0.00
4,4,4,2022-03-18 21:04:16,"Have you heard of $PIT, one of the largest com...",False,0.25
...,...,...,...,...,...,...
295,295,295,2022-01-12 09:21:10,NFT news.\n\nKakao Entertainment to expand its...,False,0.00
296,296,296,2021-04-24 12:24:23,Crypto mining as it should be! \n\nZionodes pr...,False,0.00
297,297,297,2021-07-21 04:19:31,"The state now has 1,350 active locally-acquire...",False,0.00
298,298,298,2021-10-19 17:25:28,What’s amazing it’s not just for the rich and ...,False,0.25
